In [1]:
import os
from os import path as osp
import av2
import torch
from ipdb import set_trace
from av2.utils.io import read_feather
import numpy as np
import multiprocessing as mp
import tqdm
import pickle as pkl

from utils import LABEL_ATTR, cuboid_to_vertices
from SO3 import quat_to_yaw

In [2]:
root = "D:/NTU/sensor"
output_dir = 'D:/NTU/ICLR/kitti_format'
save_bin = True

In [4]:
def prepare(root):
    ts2idx = {}
    ts_list = []
    bin_idx_list = []
    seg_path_list = []
    seg_split_list = []
    assert root.split('/')[-1] == 'sensor'
    splits = ['train', 'val', 'test']
    # splits = ['train', ]
    num_train_samples = 0
    num_val_samples = 0
    num_test_samples = 0

    # 0 for training, 1 for validation and 2 for testing.
    prefixes = [0, 1, 2]

    for i in range(len(splits)):
        split = splits[i]
        prefix = prefixes[i]
        split_root = osp.join(root, split)
        seg_file_list = os.listdir(split_root)
        print(f'num of {split} segments:', len(seg_file_list))
        for seg_idx, seg_name in enumerate(seg_file_list):
            seg_path = osp.join(split_root, seg_name)
            seg_path_list.append(seg_path)
            seg_split_list.append(split)
            assert seg_idx < 1000
            frame_path_list = os.listdir(osp.join(seg_path, 'sensors/lidar/'))
            for frame_idx, frame_path in enumerate(frame_path_list):
                assert frame_idx < 1000
                bin_idx = str(prefix) + str(seg_idx).zfill(3) + str(frame_idx).zfill(3)
                ts = frame_path.split('/')[-1].split('.')[0]
                ts = seg_name + '/' + ts # ts is not unique, so add seg_name
                ts2idx[ts] = bin_idx
                ts_list.append(ts)
                bin_idx_list.append(bin_idx)
        if split == 'train':
            num_train_samples = len(ts_list)
        elif split == 'val':
            num_val_samples = len(ts_list) - num_train_samples
        else:
            num_test_samples = len(ts_list) - num_train_samples - num_val_samples
    # print three num samples
    print('num of train samples:', num_train_samples)
    print('num of val samples:', num_val_samples)
    print('num of test samples:', num_test_samples)

    assert len(ts_list) == len(set(ts_list))
    assert len(bin_idx_list) == len(set(bin_idx_list))
    return ts2idx, seg_path_list, seg_split_list

In [5]:
ts2idx, seg_path_list, seg_split_list = prepare(root)

num of train segments: 26
num of val segments: 12
num of test segments: 12
num of train samples: 4069
num of val samples: 1878
num of test samples: 1878


In [30]:
seg_split_list

['D:/NTU/sensor\\train\\022af476-9937-3e70-be52-f65420d52703',
 'D:/NTU/sensor\\train\\0322b098-7e42-34db-bcec-9a4d072191e9',
 'D:/NTU/sensor\\train\\04973bcf-fc64-367c-9642-6d6c5f363b61',
 'D:/NTU/sensor\\train\\0526e68e-2ff1-3e53-b0f8-45df02e45a93',
 'D:/NTU/sensor\\train\\05853f69-f948-3d04-8d64-d4e721c0e1a5',
 'D:/NTU/sensor\\train\\067b1c50-6567-3840-ab56-1ca2a0ed9c30',
 'D:/NTU/sensor\\train\\06852209-b868-306b-b492-ee6dbc914cf8',
 'D:/NTU/sensor\\train\\069cc46d-38bb-309d-88cf-296a3d0c0820',
 'D:/NTU/sensor\\train\\072c8e90-a51c-3429-9cdf-4dababb4e9d8',
 'D:/NTU/sensor\\train\\0749e9e0-ca52-3546-b324-d704138b11b5',
 'D:/NTU/sensor\\train\\074d2237-ed1b-34d7-a2fc-68edbce50bb2',
 'D:/NTU/sensor\\train\\080b1ce2-9477-39ee-8233-b7f33e1dfe56',
 'D:/NTU/sensor\\train\\08734a1b-0289-3aa3-a6ba-8c7121521e26',
 'D:/NTU/sensor\\train\\087695bd-c662-3e86-83b4-aedc3b8eec36',
 'D:/NTU/sensor\\train\\087fec73-1a0c-399a-9292-cc2cf99dc97f',
 'D:/NTU/sensor\\train\\094c4119-eb33-3dfb-a18d-492cbdc

In [28]:
ts2idx

{'022af476-9937-3e70-be52-f65420d52703/315972121860610000': '0000000',
 '022af476-9937-3e70-be52-f65420d52703/315972121960143000': '0000001',
 '022af476-9937-3e70-be52-f65420d52703/315972122060322000': '0000002',
 '022af476-9937-3e70-be52-f65420d52703/315972122159855000': '0000003',
 '022af476-9937-3e70-be52-f65420d52703/315972122260051000': '0000004',
 '022af476-9937-3e70-be52-f65420d52703/315972122360247000': '0000005',
 '022af476-9937-3e70-be52-f65420d52703/315972122459780000': '0000006',
 '022af476-9937-3e70-be52-f65420d52703/315972122559977000': '0000007',
 '022af476-9937-3e70-be52-f65420d52703/315972122660173000': '0000008',
 '022af476-9937-3e70-be52-f65420d52703/315972122759706000': '0000009',
 '022af476-9937-3e70-be52-f65420d52703/315972122859902000': '0000010',
 '022af476-9937-3e70-be52-f65420d52703/315972122960098000': '0000011',
 '022af476-9937-3e70-be52-f65420d52703/315972123059614000': '0000012',
 '022af476-9937-3e70-be52-f65420d52703/315972123159810000': '0000013',
 '022a

In [16]:
segment_path=seg_path_list[0]

In [29]:
segment_path

'D:/NTU/sensor\\train\\022af476-9937-3e70-be52-f65420d52703'

In [24]:
segname = segment_path.split('/')[-1]

In [26]:
segname

'sensor\\train\\022af476-9937-3e70-be52-f65420d52703'

In [14]:
segment_anno = read_feather('D:/NTU/sensor\\train\\022af476-9937-3e70-be52-f65420d52703/annotations.feather')

In [17]:
frame_path_list = os.listdir(osp.join(segment_path, 'sensors/lidar/'))

In [18]:
frame_name=frame_path_list[0]

In [21]:
frame_name

'315972121860610000.feather'

In [19]:
ts = int(osp.basename(frame_name).split('.')[0])

In [ ]:
frame_anno = segment_anno[segment_anno['timestamp_ns'] == ts]
frame_anno

In [23]:
frame_path = osp.join(segment_path, 'sensors/lidar/', frame_name)
frame_path

'D:/NTU/sensor\\train\\022af476-9937-3e70-be52-f65420d52703\\sensors/lidar/315972121860610000.feather'

In [25]:
frame_info = {}
frame_info['uuid'] = segname + '/' + frame_path.split('/')[-1].split('.')[0]
frame_info

{'uuid': 'sensor\\train\\022af476-9937-3e70-be52-f65420d52703/315972121860610000'}

In [27]:
frame_info['sample_idx'] = ts2idx[frame_info['uuid']]

KeyError: 'sensor\\train\\022af476-9937-3e70-be52-f65420d52703/315972121860610000'

In [ ]:
def process_and_save_frame(frame_path, frame_anno, ts2idx, segname, output_dir, save_bin):
    frame_info = {}
    frame_info['uuid'] = segname + '/' + frame_path.split('/')[-1].split('.')[0]
    frame_info['sample_idx'] = ts2idx[frame_info['uuid']]
    frame_info['image'] = dict()
    frame_info['point_cloud'] = dict(
        num_features=4,
        velodyne_path=None,
    )
    frame_info['calib'] = dict() # not need for lidar-only
    frame_info['pose'] = dict() # not need for single frame
    frame_info['annos'] = dict(
        name=None,
        truncated=None,
        occluded=None,
        alpha=None,
        bbox=None, # not need for lidar-only
        dimensions=None,
        location=None,
        rotation_y=None,
        index=None,
        group_ids=None,
        camera_id=None,
        difficulty=None,
        num_points_in_gt=None,
    )
    frame_info['sweeps'] = [] # not need for single frame
    if frame_anno is not None:
        frame_anno = frame_anno[frame_anno['num_interior_pts'] > 0]
        cuboid_params = frame_anno.loc[:, list(LABEL_ATTR)].to_numpy()
        cuboid_params = torch.from_numpy(cuboid_params)
        yaw = quat_to_yaw(cuboid_params[:, -4:])
        xyz = cuboid_params[:, :3]
        wlh = cuboid_params[:, [4, 3, 5]]

        # waymo_yaw is equal to yaw
        # corners = cuboid_to_vertices(cuboid_params)
        # c0 = corners[:, 0, :]
        # c4 = corners[:, 4, :]
        # waymo_yaw = torch.atan2(c0[:, 1] - c4[:, 1], c0[:, 0] - c4[:, 0])
        yaw = -yaw - 0.5 * np.pi

        while (yaw < -np.pi).any():
            yaw[yaw < -np.pi] += 2 * np.pi

        while (yaw > np.pi).any():
            yaw[yaw > np.pi] -= 2 * np.pi
        
        # bbox = torch.cat([xyz, wlh, yaw.unsqueeze(1)], dim=1).numpy()
        
        cat = frame_anno['category'].to_numpy().tolist()
        cat = [c.lower().capitalize() for c in cat]
        cat = np.array(cat)

        num_obj = len(cat)

        annos = frame_info['annos']
        annos['name'] = cat
        annos['truncated'] = np.zeros(num_obj, dtype=np.float64)
        annos['occluded'] = np.zeros(num_obj, dtype=np.int64)
        annos['alpha'] = -10 * np.ones(num_obj, dtype=np.float64)
        annos['dimensions'] = wlh.numpy().astype(np.float64)
        annos['location'] = xyz.numpy().astype(np.float64)
        annos['rotation_y'] = yaw.numpy().astype(np.float64)
        annos['index'] = np.arange(num_obj, dtype=np.int32)
        annos['num_points_in_gt'] = frame_anno['num_interior_pts'].to_numpy().astype(np.int32)
    # frame_info['group_ids'] = np.arange(num_obj, dtype=np.int32)
    prefix2split = {'0': 'training', '1': 'training', '2': 'testing'}
    sample_idx = frame_info['sample_idx']
    split = prefix2split[sample_idx[0]]
    abs_save_path = osp.join(output_dir, split, 'velodyne', f'{sample_idx}.bin')
    rel_save_path = osp.join(split, 'velodyne', f'{sample_idx}.bin')
    frame_info['point_cloud']['velodyne_path'] = rel_save_path
    if save_bin:
        save_point_cloud(frame_path, abs_save_path)
    return frame_info


In [ ]:
def process_single_segment(segment_path, split, info_list, ts2idx, output_dir, save_bin):
    test_mode = 'test' in split
    if not test_mode:
        segment_anno = read_feather(osp.join(segment_path, 'annotations.feather'))
    segname = segment_path.split('/')[-1]

    frame_path_list = os.listdir(osp.join(segment_path, 'sensors/lidar/'))

    for frame_name in frame_path_list:
        ts = int(osp.basename(frame_name).split('.')[0])

        if not test_mode:
            frame_anno = segment_anno[segment_anno['timestamp_ns'] == ts]
        else:
            frame_anno = None

        frame_path = osp.join(segment_path, 'sensors/lidar/', frame_name)
        frame_info = process_and_save_frame(frame_path, frame_anno, ts2idx, segname, output_dir, save_bin)
        info_list.append(frame_info)

In [9]:
def main(seg_path_list, seg_split_list, info_list, ts2idx, output_dir, save_bin, token, num_process):
    for seg_i, seg_path in enumerate(seg_path_list):
        if seg_i % num_process != token:
            continue
        print(f'processing segment: {seg_i}/{len(seg_path_list)}')
        split = seg_split_list[seg_i]
        process_single_segment(seg_path, split, info_list, ts2idx, output_dir, save_bin)
        print(info_list)

['train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'val',
 'val',
 'val',
 'val',
 'val',
 'val',
 'val',
 'val',
 'val',
 'val',
 'val',
 'val',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test',
 'test']